In [ ]:
import os, pickle, sqlite3, os, math, helper
import pandas as pd
from nose.tools import assert_true, assert_equal, assert_is_instance

Let's read in the familiar car price data.

In [ ]:
df = pd.read_csv('./imports-85.data')

In [ ]:
df.head()

Now let's suppose that we only need the "make" column of this file. We don't want to keep reading in the whole file, so we will instead just pickle and save the "make" column.

In [ ]:
make = df['make'].values
df=0

# Problem 1: Pickle data and save to a file

Write a function called "save" which takes in some data that will be pickled and saves it in a file called "filename" 

In [ ]:
def save(data, filename):
    """
    Inputs
    ----------
    data: Any pickable object.
    filename: A string, the name to give the pickled data
    
    Returns
    -------
    None
    """
    
    # YOUR CODE HERE



In [ ]:
save(make, "make.pickle")

assert_true(os.path.exists("make.pickle"))

def get_checksum(filename):
    import hashlib
    with open(filename, 'rb') as f:
        md5 = hashlib.md5(f.read())
    return md5.hexdigest()

assert_equal(
    get_checksum("make.pickle"),
    '69ce5d072a0df028ee274c3106b60a63')

# Problem 2: Unpickle data

Write a function called "load" that given the argument "filename" unpickles the file.

In [ ]:
def load(filename):
    """
    Inputs
    ----------
    filename: A string containing the file name to unpickle
    
    Returns
    -------
    data: the unpickled data
    """
    
    # YOUR CODE HERE

In [ ]:
unpickled_make = load("make.pickle")

assert_equal(unpickled_make[0],0)
assert_equal(len(unpickled_make),205)

In [ ]:
#this line deletes the make.pickle file
os.remove("make.pickle")

# Problem 3: Create a schema file

In the below code cell, create a schema file called "cars.sql" with three columns: 

$\texttt{make}$: an integer

$\texttt{price}$: a real number

$\texttt{enginelocation}$: a string

Name the table "carinfo". Note that SQLite uses a different symbol that Pyhon to denote a comment. "--" denotes a comment in SQLite.

In [ ]:
%%writefile cars.sql

-- YOUR CODE HERE

# YOUR CODE HERE

In [ ]:
assert_true(os.path.exists("cars.sql"))

# Problem 4: Create the schema in a new database

In the below cell write a UNIX command that will create the schema in a new database named "cars.db"

In [ ]:
# YOUR CODE HERE

In [ ]:
#Let's check to make sure it worked, if so it will output the schema made in the cars.sql file
!sqlite3 cars.db ".schema"

In [ ]:
assert_true(os.path.exists("cars.db"))

!sqlite3 cars.db "DELETE FROM carinfo"
!sqlite3 cars.db "INSERT INTO carinfo VALUES (1,137.3,'Hyundai')"
test_result = !sqlite3 cars.db "SELECT * FROM carinfo"
assert_equal(test_result, ['1|137.3|Hyundai'])

In [ ]:
!rm cars.*

In [ ]:
df = pd.read_csv("./breast-cancer-wisconsin.data")
df.head(5)

# Problem 5: Creating a Database
For this problem:
- Create a database using sqlite3 called *breast-cancer-wisconsin* in the same directory and establish a connection to it.
- Create a table in this database called *bCancer*. All of the columns present in the breast cancer dataset data frame should be created in the database.
    - The type for all columns should be numeric, excluding the id column. Make the type of the id column text. In addition to the columns from the breast cancer dataset, also create a new column called combined.
    - When you create the table be sure to create the columns in this order with the following names (df name: col name in db):
        - id: id
        - clump thickness: c_thick
        - uniformity cell size: u_c_size
        - uniformity cell shape: u_c_shape
        - marginal adhesion: m_adh
        - epithelial cell size: e_cell_size
        - bare nuclei: b_nuceli
        - bland chromatin: b_chro
        - normal nucleoli: n_nucl
        - mitoses: mitoses
        - class: class
        - combined: combined
  
- Commit the changes and **close the connection to your database** (if you do not close the connection you may run into problems.)

In [ ]:
# YOUR CODE HERE

In [ ]:
# Making sure db Exists
assert_equal(os.path.exists("./breast-cancer-wisconsin.db"), True, "Make a database called breast-cancer-wisconsin in the same directory.")

# Making sure Table Exist
test_conn = sqlite3.connect('breast-cancer-wisconsin.db')
test_c = test_conn.cursor()
test_c.execute("SELECT count(*) FROM sqlite_master WHERE type='table' AND name='bCancer';")
sol = test_c.fetchone()[0]
assert_equal(sol, 1, "You need to a table called bCancer")

# Making sure the columns in the table match the description
test_c.execute('select * from bCancer')
names = [description[0] for description in test_c.description]
assert_equal(names, ['id', 'c_thick', 'u_c_size', 'u_c_shape', 'm_adh', 'e_cell_size', 'b_nuceli', 'b_chro', 'n_nucl', 'mitoses', 'class', 'combined'])

test_conn.close()  # Closing connection

# Problem 6: Populating a database
For this problem you will populate a database using the breast cancer data stored df (which is of type pandas dataframe). Do the following:
- You need to create data for our new column combined. The data for combined should be made by the following expression: *uniformity cell size ^2 + uniformity cell shape - sqrt(marginal adhesion)* Round the data to 3 decimal places. *There are many ways to do this*
- Open the database you have created.
- Next iterate through the pandas dataframe starting from row 0 until the end of the dataframe. For each iteration store the rows using the order defined in the problem above.
- Lastly close the connection to the database.

In [ ]:
# YOUR CODE HERE

In [ ]:
test_conn = sqlite3.connect('breast-cancer-wisconsin.db')
test_c = test_conn.cursor()
test_c.execute("Select * from bCancer")
ans = test_c.fetchall()
test_conn.close()
assert_equal(ans, helper.wbc_combined)

# Problem 7: Querying a database
For this problem do the following:
-  open a connect to the breast-cancer database
- select the clump thickness, mitoses, combined , and class column *(in this order)* from the bCancer table but only select rows where combined is greater than 50. Stored this result in a variable called *ans* and close the connection to your database.

In [ ]:
# YOUR CODE HERE

In [ ]:
assert_equal(ans, helper.wbc_sol50)

# Problem 8: Dropping Tables
For this problem do the following:
- open a connection to the breast-cancer database we have worked with so far.
- Now drop the bCancer table
- Commit the changes
- Close the connection to the breast-cancer database


In [ ]:
# YOUR CODE HERE

In [ ]:
test_c.execute("SELECT count(*) FROM sqlite_master WHERE type='table' AND name='bCancer';")
sol = test_c.fetchone()[0]
assert_equal(sol, 0, "You need delete the bCancer table")

In [ ]:
! rm breast-cancer-wisconsin.db